# Import Library

In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
from utils.preprocessing import cleaningText
from utils.preprocessing import casefoldingText
from utils.preprocessing import stemmingText
from utils.preprocessing import filteringText
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load model
model = SentenceTransformer('distiluse-base-multilingual-cased')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\ASUS\Desktop\arkanesia-dataset\arkanesia_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [4]:
wisata_url = "https://raw.githubusercontent.com/novaandini/arkanesia/refs/heads/main/data/wisata_indonesia.csv"

wisata_df = pd.read_csv(wisata_url)
wisata_df.head()

,name,address,category,city,regency,province,description,keywords,price
0,Pantai Lampuuk,"Lampuuk, Lhoknga",Wisata Bahari,Banda Aceh,Aceh Besar,Aceh,Pantai indah dengan pasir putih dan ombak yang...,"Pantai, Lampuuk, Lhoknga, Aceh, Surfing",0
1,Pulau Weh,"Pulau Weh, Sabang",Wisata Bahari,Sabang,Sabang,Aceh,Pulau yang terkenal dengan keindahan alam bawa...,"Pulau, Weh, Sabang, Wisata Bahari, Diving",0
2,Pantai Ujong Blang,"Ujong Blang, Lhokseumawe",Wisata Bahari,Lhokseumawe,Aceh Utara,Aceh,Pantai yang cocok untuk bersantai dengan peman...,"Pantai, Ujong Blang, Lhokseumawe, Aceh Utara",0
3,Pantai Kuala Merisi,"Kuala Merisi, Aceh Timur",Wisata Bahari,Langsa,Aceh Timur,Aceh,Pantai yang menawarkan keindahan alam dan kete...,"Pantai, Kuala Merisi, Langsa, Aceh Timur",0
4,Pantai Lagoi,"Lagoi, Bener Meriah",Wisata Bahari,Bener Meriah,Aceh,Aceh,Pantai eksotis dengan air laut yang jernih dan...,"Pantai, Lagoi, Bener Meriah, Aceh",0


# Explore Data

In [5]:
wisata_df.duplicated().sum()

np.int64(0)

In [6]:
wisata_df.isnull().sum()

name             0
address         49
category       430
city            17
regency          0
province         0
description      0
keywords         0
price            0
dtype: int64

In [7]:
wisata_df.describe()

,name,address,category,city,regency,province,description,keywords,price
count,5875,5826,5445,5858,5875,5875,5875,5875,5875
unique,5253,3794,336,589,389,47,5790,5257,166
top,Taman Makam Pahlawan,"Merauke, Papua Selatan",Tempat Bermain,Sorong,Manokwari,Aceh,Banten,"gunung, pendakian, alam",0
freq,10,177,526,216,203,228,36,23,2812


In [8]:
wisata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5875 entries, 0 to 5874
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5875 non-null   object
 1   address      5826 non-null   object
 2   category     5445 non-null   object
 3   city         5858 non-null   object
 4   regency      5875 non-null   object
 5   province     5875 non-null   object
 6   description  5875 non-null   object
 7   keywords     5875 non-null   object
 8   price        5875 non-null   object
dtypes: object(9)
memory usage: 413.2+ KB


# Handle Null Data

In [10]:
wisata_df['region'] = wisata_df['regency']
wisata_df.drop(['category', 'city', 'regency'], axis=1, inplace=True)


KeyError: 'regency'

In [13]:
wisata_df.to_csv("../data/wisata.csv", index=True)

In [88]:
wisata_df['address'] = wisata_df['address'].fillna('unknown')

In [89]:
wisata_df.isnull().sum()

name           0
address        0
province       0
description    0
keywords       0
price          0
region         0
dtype: int64

# Preprocessing Data

## Cleaning Data

In [90]:
wisata_cleaned_df = pd.DataFrame(wisata_df)
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(cleaningText)
wisata_cleaned_df['keywords'] = wisata_cleaned_df['keywords'].apply(cleaningText)
wisata_cleaned_df['region'] = wisata_cleaned_df['region'].apply(cleaningText)
wisata_cleaned_df['province'] = wisata_cleaned_df['province'].apply(cleaningText)

## Case Folding Data

In [91]:
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(casefoldingText)
wisata_cleaned_df['keywords'] = wisata_cleaned_df['keywords'].apply(casefoldingText)
wisata_cleaned_df['region'] = wisata_cleaned_df['region'].apply(casefoldingText)
wisata_cleaned_df['province'] = wisata_cleaned_df['province'].apply(casefoldingText)
wisata_cleaned_df.head()

,name,address,province,description,keywords,price,region
0,Pantai Lampuuk,"Lampuuk, Lhoknga",aceh,pantai indah dengan pasir putih dan ombak yang...,pantai lampuuk lhoknga aceh surfing,0,aceh besar
1,Pulau Weh,"Pulau Weh, Sabang",aceh,pulau yang terkenal dengan keindahan alam bawa...,pulau weh sabang wisata bahari diving,0,sabang
2,Pantai Ujong Blang,"Ujong Blang, Lhokseumawe",aceh,pantai yang cocok untuk bersantai dengan peman...,pantai ujong blang lhokseumawe aceh utara,0,aceh utara
3,Pantai Kuala Merisi,"Kuala Merisi, Aceh Timur",aceh,pantai yang menawarkan keindahan alam dan kete...,pantai kuala merisi langsa aceh timur,0,aceh timur
4,Pantai Lagoi,"Lagoi, Bener Meriah",aceh,pantai eksotis dengan air laut yang jernih dan...,pantai lagoi bener meriah aceh,0,aceh


### Stemming Text

In [92]:
wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(stemmingText)
wisata_cleaned_df.head()

,name,address,province,description,keywords,price,region
0,Pantai Lampuuk,"Lampuuk, Lhoknga",aceh,pantai indah dengan pasir putih dan ombak yang...,pantai lampuuk lhoknga aceh surfing,0,aceh besar
1,Pulau Weh,"Pulau Weh, Sabang",aceh,pulau yang kenal dengan indah alam bawah laut ...,pulau weh sabang wisata bahari diving,0,sabang
2,Pantai Ujong Blang,"Ujong Blang, Lhokseumawe",aceh,pantai yang cocok untuk santa dengan pandang l...,pantai ujong blang lhokseumawe aceh utara,0,aceh utara
3,Pantai Kuala Merisi,"Kuala Merisi, Aceh Timur",aceh,pantai yang tawar indah alam dan tenang ideal ...,pantai kuala merisi langsa aceh timur,0,aceh timur
4,Pantai Lagoi,"Lagoi, Bener Meriah",aceh,pantai eksotis dengan air laut yang jernih dan...,pantai lagoi bener meriah aceh,0,aceh


### Filtering text

In [93]:
# wisata_cleaned_df['description'] = wisata_cleaned_df['description'].apply(filteringText)
# wisata_cleaned_df.head()

## Buat Embeddings

In [94]:
wisata_cleaned_df['combined'] = (
    wisata_cleaned_df['name'] + ' ' +
    wisata_cleaned_df['region'] + ' ' +
    wisata_cleaned_df['province'] + ' ' +
    wisata_cleaned_df['keywords'] + ' ' +
    wisata_cleaned_df['description']
)

In [95]:
# Buat embeddings
embeddings = model.encode(wisata_cleaned_df['combined'].tolist(), convert_to_tensor=True)

In [126]:
def rekomendasi_wisata(pertanyaan_user, top_k=3):
    # 1. Simpan pertanyaan asli untuk pengecekan lokasi
    original_question = pertanyaan_user.lower()

    # 2. Optional: list provinsi dari dataset
    all_provinces = wisata_cleaned_df['province'].str.lower().unique()

    # 3. Cek apakah pertanyaan mengandung nama provinsi
    lokasi_filter = None
    for prov in all_provinces:
        if prov in original_question:
            lokasi_filter = prov
            break

    # 4. Filter berdasarkan lokasi (jika ada)
    if lokasi_filter:
        df_filtered = wisata_cleaned_df[wisata_cleaned_df['province'].str.lower().str.contains(lokasi_filter)]
        if df_filtered.empty:
            print(f"Tidak ada data wisata di lokasi '{lokasi_filter.title()}', menampilkan hasil umum...\n")
            df_filtered = wisata_cleaned_df
    else:
        df_filtered = wisata_cleaned_df

    # 5. Preprocessing pertanyaan user
    pertanyaan_user = cleaningText(pertanyaan_user)
    pertanyaan_user = casefoldingText(pertanyaan_user)
    pertanyaan_user = stemmingText(pertanyaan_user)

    # 6. Embedding pertanyaan user
    query_embedding = model.encode(pertanyaan_user, convert_to_tensor=True)

    # 7. Embedding destinasi dari dataframe terfilter
    doc_embeddings = model.encode(df_filtered['combined'].tolist(), convert_to_tensor=True)

    # 8. Hitung cosine similarity
    cos_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]

    # 9. Ambil top_k hasil
    top_results = cos_scores.topk(k=top_k)

    # 10. Tampilkan hasil
    for score, idx in zip(top_results[0], top_results[1]):
        idx = idx.item()
        row = df_filtered.iloc[idx]
        print(f"{row['name']} ({row['province']}) - Skor: {score:.4f}")
        print(f"Deskripsi: {row['description']}")
        print("-" * 50)


In [127]:
rekomendasi_wisata("pantai di provinsi bangka belitung")

Pantai Penyabong (bangka belitung) - Skor: 0.4474
Deskripsi: pantai yang indah dengan giat main air dan nikmat indah alam sekitar
--------------------------------------------------
Pantai Tanjung Bemban (bangka belitung) - Skor: 0.4238
Deskripsi: pantai yang tenang ideal untuk main pasir dan renang sama keluarga
--------------------------------------------------
Pantai Bukit Berahu (bangka belitung) - Skor: 0.4209
Deskripsi: pantai dengan suasana tenang cocok untuk renang dan main pasir putih
--------------------------------------------------
